# 1D Monitoring Element Exploration

This notebook is intended to showcase `dqmexplore`'s 1D monitoring element (ME) exploration tools.

## Setup

If you are working in SWAN, you will need to install the `omsapi` as well as `dqmexplore` itself.

In [ ]:
# Run this if you are in SWAN
# Package installation/updating
!pip3 install -e .. --no-dependencies
!pip3 install omsapi
!pip3 install cmsdials --upgrade

### Imports

In [ ]:
# DIALS API
# For more information on DIALS, please visit https://github.com/cms-DQM/dials-py
import cmsdials
from cmsdials.auth.client import AuthClient
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import LumisectionHistogram1DFilters, LumisectionHistogram2DFilters

auth = AuthClient()
token = auth.device_auth_flow()
creds = Credentials.from_authclient_token(token)
dials = Dials(creds)

Run your prefered authentication method for the OMS API. For more information on the OMS API, please visit https://gitlab.cern.ch/cmsoms/oms-api-client

In [ ]:
# Method 1: krb
import omsapi

oms_fetch = omsapi.OMSAPI("https://cmsoms.cern.ch/agg/api", "v1", cert_verify=False)
oms_fetch.auth_krb()

In [ ]:
# Method 2: oidc
import omsapi
import json

with open("./clientid.json", "r") as file:
    secrets = json.load(file)

oms_fetch = omsapi.OMSAPI("http://vocms0185.cern.ch/agg/api", "v1", cert_verify=False)
oms_fetch.auth_oidc(secrets["API_CLIENT_ID"], secrets["API_CLIENT_SECRET"], audience="cmsoms-int-0185")

del(secrets)
del(file)

In [ ]:
# DQMExplore
import dqmexplore
from dqmexplore.plotting import plot1DMEs
from dqmexplore.exploreutils import check_empty_lss
from dqmexplore.dataproc import generate_me_dict
from dqmexplore.statplotting import plotheatmaps1D, plotMEs1D_static
from dqmexplore.omsutils import get_rate, plot_rate
from dqmexplore.trends import compute_trends, plot_trends 

# Plotly
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

# Other useful libraries for data exploration, manipulation and processing
import numpy as np
import pandas as pd

If you run the following, you will get a list of all the available 1D MEs.

In [ ]:
mes_df = pd.DataFrame([me_qry_rslt.__dict__ for me_qry_rslt in dials.mes.list()])
mes_df[mes_df["dim"] == 1]

## Using OMS to Obtain Metadata

Using the OMS API, we can access important information regarding the run conditions and other information about the run. The available endpoints are:

* `lumisections`
* `runs`
* `fills`
* `datasetrates`

You can access the trigger rate* in the following way:

*HLT ZeroBias trigger rate

In [ ]:
runnb = 380238
omstrig_df = get_rate(oms_fetch, runnb, "ZeroBias")
fig = plot_rate(omstrig_df, norm=False, show=True)

# Optionally, you can export the Plotly figure object dqmexplore returns. Just remember to set show=False in plot_rate
# plot(fig, filename=f"./plots/trigrate_{runnb}.html")

## 1D Monitoring Elements

We fetch data from DIALS as shown here. For more information on how to use the DIALS Python API, please refer to the [official repository](https://github.com/cms-DQM/dials-py). If you are unfamiliar with regex syntax, you can take a look at the following [cheat sheet](https://www.rexegg.com/regex-quickstart.html) for a quick overview.

In [ ]:
# Getting current run data
runnb = 380238
me__regex =  "PixelPhase1/Tracks/PXBarrel/charge_PXLayer_." 

data1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    # max_pages=200
).to_pandas()

In [ ]:
# Getting current run trigger rate
trig_rate = get_rate(oms_fetch, runnb, "ZeroBias", dataframe=False)

In [ ]:
# Checking for empty LSs, empty here meaning that the number of entries in the historgram is less than the set threshold (default=10)
print("Empty/Near empty LSs:")
pd.DataFrame(check_empty_lss(generate_me_dict(data1D))).T

In [ ]:
# Getting reference data
refrun = 379765

refdata1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = refrun,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    # max_pages=200
).to_pandas()

In [ ]:
# Defining plot features
ax_labels = [
    dict(
        x="Charge (e)", 
        y="Count"
    )
] * 4

fig_title = f"Pixel Barrel Charge Normalized (Run {runnb})"

# Plotting
fig = plot1DMEs(
    data1D, 
    fig_title=fig_title,
    ax_labels=ax_labels, 
    width=1000,
    height=1000,
    vspace=0.1,
    trigger_rates=trig_rate, 
    # norm=True,
    # ref_df=refdata1D
    show=True
)

# Export plot to html. Remember to set show=False for plot1DMEs
# plot(fig, filename=f"./plots/PixelBarrelCharge-run{runnb}-ref{refrun}-normalized.html")

### Heatmaps

By "stacking" 1D histograms, we can create heatmaps which give us an idea of how the run evolved through time as data was being taken.

In [ ]:
# Getting current run data
runnb = 380238
me__regex =  "PixelPhase1/Tracks/PXBarrel/charge_PXLayer_." 

data1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    # max_pages=200
).to_pandas()

In [ ]:
# Getting trigger rate
trig_rate = get_rate(oms_fetch, runnb, "ZeroBias", dataframe=False)

In [ ]:
ax_labels = [
    dict(x = "Charge(e)", y = "LS")
] * 4

fig_title = f"Pixel Barrel Charge Heatmaps Normalized by Trigger Rate (Run {runnb})"

fig = plotheatmaps1D(
    data1D,
    fig_title=fig_title,
    ax_labels=ax_labels,
    show=False,
    trigger_rates = trig_rate,
    # norm=True
)

fig.show()

# Export plot to html
# plot(fig, filename=f"./plots/PixelBarrelChargeHeatmap-run{runnb}-trignorm.html")